# Imports

In [60]:
import pandas as pd
from unidecode import unidecode
import dtale

# Load Data

In [61]:
df_gupy = pd.read_excel('data_raw/vagas_gupy.xlsx')

df_vagas = pd.concat([df_gupy], axis = 0)
df_vagas.head()

,site_da_vaga,link,data_publicacao,data_coleta,titulo_da_vaga,local,modalidade,contrato,acessibilidade,nome_empresa,codigo_vaga,descricao,regime
0,Gupy,https://vtal.gupy.io/job/eyJqb2JJZCI6NjU4NzUyN...,Publicada em: 19/01/2024,2024-01-20,Analista de Dados II,Rio de Jan... - RJ,Presencial,Efetivo,Também p/ PcD,V.tal | o futuro passa por aqui,NaN,Descrição da vagaO profissional fará parte do ...,NaN
1,Gupy,https://celcoin.gupy.io/job/eyJqb2JJZCI6NjIyOD...,Publicada em: 19/01/2024,2024-01-20,Analista de Dados Jr.,Barueri - SP,Híbrido,Efetivo,Também p/ PcD,Celcoin,NaN,Descrição da vagaFaça parte de uma Startup ino...,NaN
2,Gupy,https://gruporpn.gupy.io/job/eyJqb2JJZCI6NjY0O...,Publicada em: 19/01/2024,2024-01-20,Grupo RPN | Analista de Inteligência de Dados ...,Americana - SP,Presencial,Efetivo,Também p/ PcD,Grupo RPN,NaN,Descrição da vagaA pessoa Analista de Inteligê...,NaN
3,Gupy,https://stefanini.gupy.io/job/eyJqb2JJZCI6NjY0...,Publicada em: 18/01/2024,2024-01-20,ANALISTA DADOS SR,Não informado,Remoto,Efetivo,Também p/ PcD,Stefanini Group,NaN,Descrição da vagaSomos a maior multinacional b...,NaN
4,Gupy,https://jbs.gupy.io/job/eyJqb2JJZCI6NjY0MDcxOC...,Publicada em: 18/01/2024,2024-01-20,Analista de Planejamento - Análise de Dados - ...,São Paulo - SP,Presencial,Efetivo,NaN,JBS,NaN,Descrição da vagaA JBS é a maior empresa de al...,NaN


# Features

## Senioridade

In [62]:
# matches para as senioridades
junior_matches = ['junior', ' jr', ' i', ' l']
pleno_matches = ['pleno', ' pl', ' ii', ' ll']
senior_matches = ['senior', ' sr', ' iii', 'lll']

# removendo acentos e deixando letras minúsculas
df_vagas['titulo_da_vaga_tratado'] = df_vagas['titulo_da_vaga'].apply(lambda x: unidecode(x.lower()))

# função de faz matches com as senioridades das vagas
def busca_senioridade(titulo_vaga, match_list):
    senioridade = False
    for sub_string in match_list:
        if sub_string in titulo_vaga:
            senioridade = True
    
    return senioridade

# aplicando função
df_vagas['senioridade_junior'] = df_vagas['titulo_da_vaga_tratado'].apply(lambda titulo_vaga: busca_senioridade(titulo_vaga, junior_matches))
df_vagas['senioridade_pleno'] = df_vagas['titulo_da_vaga_tratado'].apply(lambda titulo_vaga: busca_senioridade(titulo_vaga, pleno_matches))
df_vagas['senioridade_senior'] = df_vagas['titulo_da_vaga_tratado'].apply(lambda titulo_vaga: busca_senioridade(titulo_vaga, senior_matches))

# categorizando senioridades
df_vagas['senioridade'] = df_vagas[['senioridade_junior', 
                                    'senioridade_pleno', 
                                    'senioridade_senior']].apply(lambda x:  'Pleno/Sênior' if (x['senioridade_pleno']) and (x['senioridade_senior']) else
                                                                            'Júnior/Pleno' if (x['senioridade_junior']) and (x['senioridade_pleno']) else
                                                                            'Júnior' if x['senioridade_junior'] else 
                                                                            'Pleno' if x['senioridade_pleno'] else
                                                                            'Sênior' if x['senioridade_senior'] else
                                                                            'Não informado'
                                                                            , axis = 1)

In [63]:
df_vagas.loc[df_vagas['senioridade'] == 'Não informado', 'titulo_da_vaga_tratado'].unique()

array(['analista de dados  (copia)', 'analista de ciencia de dados ',
       'cas | analista de dados - foco em gestao de comunidade de dados',
       'analista de dados ',
       'analista de dados - geoprocessamento | manufacturing.',
       'analista de dados (remoto)', 'analista de dados | salvador',
       'analista de dados (tableau)', 'analista de dados | mercado ',
       'analista de dados',
       'analista de privacidade de dados (lgpd) - presencial brasilia',
       'analista de customer service - dados | ribas do rio pardo/ms',
       'logistica: analista de dados', 'analista de dados - comercial   ',
       'analista de dados - jundiai/sp ', 'analista de dados comerciais',
       'analista dados ti',
       'analista de dados comerciais - direcionado a vendas',
       'analista contabil - analista de dados ',
       'analista de marketing - crm (dados)',
       'cas | analista de dados - foco em associacao e contas',
       'analista de dados | cresol confederacao',
     

## Localidade

In [64]:
df_vagas['cidade'] = df_vagas['local'].apply(lambda x: 'Não informado' if x == 'Não informado' else x.split(' - ')[0])
df_vagas['estado'] = df_vagas['local'].apply(lambda x: 'Não informado' if x == 'Não informado' else x.split(' - ')[1])

In [65]:
df_vagas['cidade']

0      Rio de Jan...
1            Barueri
2          Americana
3      Não informado
4          São Paulo
           ...      
191    Rio de Jan...
192        São Paulo
193    Rio de Jan...
194    Não informado
195        São Paulo
Name: cidade, Length: 196, dtype: object

# Save Dataframe

In [66]:
columns_selected = [
'site_da_vaga',
 'link',
 'data_publicacao',
 'data_coleta',
 'titulo_da_vaga',
 'senioridade',
 'cidade',
 'estado',
 'modalidade',
 'contrato',
 'acessibilidade',
 'nome_empresa',
 'codigo_vaga',
 'descricao'
]

In [67]:
df_vagas[columns_selected].to_excel('vagas_clean.xlsx', index=False)

In [68]:
df_vagas[columns_selected]

,site_da_vaga,link,data_publicacao,data_coleta,titulo_da_vaga,senioridade,cidade,estado,modalidade,contrato,acessibilidade,nome_empresa,codigo_vaga,descricao
0,Gupy,https://vtal.gupy.io/job/eyJqb2JJZCI6NjU4NzUyN...,Publicada em: 19/01/2024,2024-01-20,Analista de Dados II,Júnior/Pleno,Rio de Jan...,RJ,Presencial,Efetivo,Também p/ PcD,V.tal | o futuro passa por aqui,NaN,Descrição da vagaO profissional fará parte do ...
1,Gupy,https://celcoin.gupy.io/job/eyJqb2JJZCI6NjIyOD...,Publicada em: 19/01/2024,2024-01-20,Analista de Dados Jr.,Júnior,Barueri,SP,Híbrido,Efetivo,Também p/ PcD,Celcoin,NaN,Descrição da vagaFaça parte de uma Startup ino...
2,Gupy,https://gruporpn.gupy.io/job/eyJqb2JJZCI6NjY0O...,Publicada em: 19/01/2024,2024-01-20,Grupo RPN | Analista de Inteligência de Dados ...,Júnior,Americana,SP,Presencial,Efetivo,Também p/ PcD,Grupo RPN,NaN,Descrição da vagaA pessoa Analista de Inteligê...
3,Gupy,https://stefanini.gupy.io/job/eyJqb2JJZCI6NjY0...,Publicada em: 18/01/2024,2024-01-20,ANALISTA DADOS SR,Sênior,Não informado,Não informado,Remoto,Efetivo,Também p/ PcD,Stefanini Group,NaN,Descrição da vagaSomos a maior multinacional b...
4,Gupy,https://jbs.gupy.io/job/eyJqb2JJZCI6NjY0MDcxOC...,Publicada em: 18/01/2024,2024-01-20,Analista de Planejamento - Análise de Dados - ...,Pleno,São Paulo,SP,Presencial,Efetivo,NaN,JBS,NaN,Descrição da vagaA JBS é a maior empresa de al...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Gupy,https://infotecbrasil.gupy.io/job/eyJqb2JJZCI6...,Publicada em: 26/12/2022,2024-01-20,Analista de Dados - Home Office,Não informado,Rio de Jan...,RJ,Remoto,Efetivo,Também p/ PcD,Infotec Brasil,NaN,Descrição da vagaExecutar atividades de suport...
192,Gupy,https://gruposkytech&76015&inactive.gupy.io/jo...,Publicada em: 31/10/2022,2024-01-20,Analista de dados,Não informado,São Paulo,SP,Presencial,Efetivo,NaN,Grupo Skytech,NaN,NaN
193,Gupy,https://brivia.gupy.io/job/eyJqb2JJZCI6OTQ5ODA...,Publicada em: 08/06/2021,2024-01-20,Analista de Dados,Não informado,Rio de Jan...,RJ,Remoto,Banco de Talentos,Também p/ PcD,Brivia,NaN,Descrição da vagaVaga para nosso BANCO DE TALE...
194,Gupy,https://queroserglobalsys.gupy.io/job/eyJqb2JJ...,Publicada em: 04/06/2021,2024-01-20,Banco de Talentos - Analista de Dados/ Analist...,Não informado,Não informado,Não informado,Banco de Talentos,Também p/ PcD,NaN,Globalsys Soluções em TI,NaN,NaN


In [69]:
d = dtale.show(df_vagas)
d.open_browser()

In [19]:
df_vagas[['senioridade', 'link']].groupby('senioridade').count().reset_index().sort_values('link', ascending=False)

,senioridade,link
2,Não informado,70
0,Júnior,46
5,Sênior,37
3,Pleno,32
4,Pleno/Sênior,6
1,Júnior/Pleno,5
